In [30]:
from helper import *
import json
from langchain_openai import ChatOpenAI
import datetime
import json
import sqlite3
from langchain_openai import ChatOpenAI
import inspect
import nltk

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [16]:
conversation_stack = [
  {
    "id": 0,
    "actor": "system",
    "message_type": "memory_instructions",
    "content": "This is a Clinic Appointment Scheduling service for a medical clinic. We can help you schedule a new appointment or reschedule an existing appointment. Whenever you believe it is appropriate, mention that the user may choose to quit the service at any time by just saying so."
  },
  {
    "message_id": 1,
    "actor": "system",
    "message_type": "system_message",
    "content": "Welcome to our Clinic Appointment Scheduling service. What would you like to do? You may quit this interface by just saying so.",
    "in_response_to": 0,
    "intent": None,
    "from_control_function": "intent_loop",
    "timestamp": "2024-07-23 15:40:26"
  },
  {
    "message_id": 2,
    "actor": "user",
    "message_type": "user_message",
    "content": "my name is Sam King and I'd like to schedule an appointment",
    "in_response_to": 1,
    "intent": None,
    "from_control_function": "intent_loop",
    "timestamp": "2024-07-23 15:40:36"
  },
  {
    "message_id": 3,
    "actor": "system",
    "message_type": "system_message",
    "content": "It sounds like you would like to schedule a new appointment. Is this correct?",
    "in_response_to": 2,
    "intent": None,
    "from_control_function": "intent_loop",
    "timestamp": "2024-07-23 15:40:37"
  },
  {
    "message_id": 4,
    "actor": "user",
    "message_type": "user_message",
    "content": "yes",
    "in_response_to": 3,
    "intent": None,
    "from_control_function": "intent_loop",
    "timestamp": "2024-07-23 15:40:39"
  },
  {
    "message_id": 5,
    "actor": "system",
    "message_type": "system_message",
    "content": "Great, let's schedule a new appointment",
    "in_response_to": 4,
    "intent": None,
    "from_control_function": "intent_loop",
    "timestamp": "2024-07-23 15:40:39"
  }
]

In [4]:
payload = {
            "first name": None,
            "last name": None,
            "reason_for_visit": None,
            "date": None,
            "time_start": None,
            "time_end": None,
        }

# interpret response

In [47]:
import dateparser

v = dateparser.parse("feb 2 1999 to march 2 1999")

print(v)

None


In [41]:
from agent import Agent
from dateparser import parse
import datetime

extract_date_instructions = [
    "You are an agent responsible for parsing strings and extracting the relevant information from them.",
    "Extract just the date from the given string",
    "Your response should be a tuple in the form on (type of date, [date 1, date 2])",
    "If the string contains a date range, return both dates",
    "Type of date can be either 'single' or 'range'",
    "The date today is " + str(datetime.datetime.now().date()),
    "The current weekday (using datetime weekday) is " + str(datetime.datetime.now().weekday())
]

date_extraction_agent = Agent(extract_date_instructions)

def extract_date(text):
    resp = eval(date_extraction_agent.call(text))
    date_type = resp[0]
    if date_type == "single":
        return parse(resp[1][0]).date()
    else:
        return False

extract_time_instructions = [
    "You are an agent responsible for parsing strings and extracting the relevant information from them.",
    "Extract just the time from the given string",
    "Your response should be a tuple in the form on (type of time, [time 1, time 2])",
    "If the string contains a time range, return both times",
    "Type of time can be either 'single' or 'range'"
    "If no time could be determined, set the type to 'none' and return an empty list",
]

time_extraction_agent = Agent(extract_time_instructions)

def extract_time(text):
    resp = eval(time_extraction_agent.call(text))
    time_type = resp[0]
    if time_type == "single":
        return parse(resp[1][0]).time()
    else:
        return False


2024-09-06
03:30:00


In [31]:
str(datetime.datetime.today().weekday())

'2'

In [54]:
# def cast_value(output_data_type, data_validation_response, debug=False):
#     if debug:
#         print(f"cast_value called with output_data_type: {output_data_type}, data_validation_response: {data_validation_response}")
#     try:
#         if output_data_type == "datetime.datetime":
#             value = datetime.datetime.strptime(data_validation_response, "%Y-%m-%d %H:%M:%S")
#         elif output_data_type == "datetime.date":
#             value = datetime.datetime.strptime(data_validation_response, "%Y-%m-%d").date()
#         elif output_data_type == "datetime.time":
#             value = datetime.datetime.strptime(data_validation_response, "%H:%M:%S").time()
#         else:
#             value = eval(output_data_type)(data_validation_response)
#         if debug:
#             print(f"Cast value: {value}")
#         return value
#     except Exception as e:
#         if debug:
#             print(f"Error casting value: {e}")
#         raise ValueError(f"Error casting value: {e}")

def cast_value(output_data_type, data_validation_response, debug=False):
    if debug:
        print(f"cast_value called with output_data_type: {output_data_type}, data_validation_response: {data_validation_response}")
    try:
        if output_data_type == "datetime.datetime":
            value = datetime.datetime.strptime(data_validation_response, "%Y-%m-%d %H:%M:%S")
        elif output_data_type == "datetime.date":
            value = extract_date(data_validation_response)
        elif output_data_type == "datetime.time":
            value = extract_time(data_validation_response)
        else:
            value = eval(output_data_type)(data_validation_response)
        if debug:
            print(f"Cast value: {value}")
        return value
    except Exception as e:
        if debug:
            print(f"Error casting value: {e}")
        raise ValueError(f"Error casting value: {e}")

def interpret_response(original_question: str, output_data_type: str, output_options, user_response: str, debug=False):
    if debug:
        print(f"interpret_response called with original_question: '{original_question}', output_data_type: '{output_data_type}', output_options: {output_options}, user_response: '{user_response}'")
    if output_options is None:
        output_options = "Can be anything"
    else:
        output_options = ', '.join(map(str, output_options))

    if output_data_type == "datetime.time" or  output_data_type == "datetime.date":
        data_validation_response = user_response
    
    else:
        prompt = (
            f"Given the following question and user response, please extract the relevant value.\n\n"
            f"Question: {original_question}\n"
            f"Expected data type: {output_data_type}\n"
            f"Possible options: {output_options}\n"
            f"User response: {user_response}\n\n"
            f"Please provide the extracted value (if possible options are given, make sure that the value exists in the possible options or return 'Not applicable'):"
            f"If the output_data_type is datetime.date or datetime.time, pleae just provide the original user response"
        )
        
        response = llm.invoke(prompt).content.strip()

        if debug:
            print(f"LLM response for interpret_response: {response}")

        data_validation_prompt = f""" An LLM has been given a task of data extraction from a string. Your job is to verify the data was extracted correctly.
        You will also be given a data type and a list of possible options for the data (can be None). Make sure that your response fits the data type and is in one of the options.
        Here is the prompt to the LLM: {prompt}
        Here is the response from the LLM: {response}
        Here is the data type: {output_data_type}
        Here are the possible options: {output_options}
        Please make sure that the response is correct and fits the data type and is in the possible options. You MUST provide a output that matches the data type exactly.
        Remember, if the output_data_type is datetime.date or datetime.time, pleae just provide the original user response.
        """
        
        data_validation_response = llm.invoke(data_validation_prompt).content.strip()

    if debug:
        print(f"Data validation response: {data_validation_response}")

    try:
        value = cast_value(output_data_type, data_validation_response, debug=debug)
        if debug:
            print(f"Interpreted value: {value}")
        return value
    except Exception as e:
        if debug:
            print(f"Error in interpret_response: {e}")
        return False


In [68]:
v = (interpret_response(original_question="When would you like to schedule the appointment on December 6th?", 
                   output_data_type="datetime.time", 
                   output_options=None, 
                   user_response="lets do it at 4 30 pm pls", 
                   debug=True))

print(v)

if v:
    print("hey")

interpret_response called with original_question: 'When would you like to schedule the appointment on December 6th?', output_data_type: 'datetime.time', output_options: None, user_response: 'lets do it at 4 30 pm pls'
Data validation response: lets do it at 4 30 pm pls
cast_value called with output_data_type: datetime.time, data_validation_response: lets do it at 4 30 pm pls
Cast value: 16:30:00
Interpreted value: 16:30:00
16:30:00
hey


In [41]:
original_question = "What is your last name, Larry?"
output_data_type = "str"
output_options = None
user_response = "I would love to schedule it July 3rd, 2024"

interpreted_value = interpret_response(original_question, output_data_type, output_options, user_response)
print(f"The interpreted value is: {interpreted_value}")

Interpreted response: July 3rd, 2024
Data validation response: 2024-07-03
The interpreted value is: 2024-07-03


# sentiment scoring

In [ ]:
def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']

def process_llm_output(conversation_stack, llm_prompt):
    llm_output = prompt_llm_with_context(conversation_stack, llm_prompt)
    
    sentiment_score = analyze_sentiment(llm_output)
    print(f"Sentiment score: {sentiment_score}")
    threshold = 0.05  # You can adjust this threshold based on your needs

    if sentiment_score >= threshold:
        curr_message = "Great, let's schedule a new appointment"
        # record_context('system', curr_message)
        # user_intent = INTENTS[0]
    else:
        curr_message = "What would you like to do instead?"
        # record_context('system', curr_message)
        curr_resp = input(curr_message + '\n')
        # record_context('user', curr_resp)
        process_llm_output(conversation_stack, llm_prompt)

# Example usage
  # Your conversation stack here
llm_prompt = "Your LLM prompt here"
process_llm_output(conversation_stack, llm_prompt)


# Logic Blocks

In [1]:
from helper import *
import json
from langchain_openai import ChatOpenAI
import datetime
import requests
import os
from agent import Agent

In [17]:
CONTEXT = {
    'intent': "schedule appointment",
    'date': "2024-07-23",
    'time_start': "09:00:00",
    'provider': {
        'provider_id': 100011110,
        'provider_first_name': "George",
        'provider_last_name': "Smith"
    }
}

AVAILABLE_APIS = {
    "schedule_api": {
        "api_name": "schedule visit",
        "api_path": "api.healion.com/practice_mgmt/schedule",
        "json_arguments": {
            "first name": None,
            "last name": None,
            "reason_for_visit": None,
            "date": None,
            "time_start": None,
            "duration": None,
            "provider": None
        },
        "json_arguments_schema": {
            "first name": "str",
            "last name": "str",
            "reason_for_visit": "str",
            "date": "datetime.date",
            "time_start": "datetime.time",
            "duration": "datetime.time",
            "provider": "dict"
        },
        "json_arguments_options": {
            "first name": None,
            "last name": None,
            "reason_for_visit": None,
            "date": None,
            "time_start": None,
            "duration": None,
            "provider": {
                "provider_id": None,
                "provider_first_name": None,
                "provider_last_name": None
            }
        },
        "json_arguments_expected_format": {
            "first name": "John",
            "last name": "Doe",
            "reason_for_visit": "checkup",
            "date": "2024-07-23",
            "time_start": "09:00:00",
            "duration": "00:30:00",
            "provider": {
                "provider_id": 100011110,
                "provider_first_name": "George",
                "provider_last_name": "Smith"
            }
        },
        "json_arguments_subgraph": {
            "provider": {
                "result_keys":
                [
                    "provider_id",
                    "provider_first_name",
                    "provider_last_name"
                ],
            }
        }
    },
    "reschedule_api": {
        "api_name": "reschedule visit",
        "api_path": "api.healion.com/practice_mgmt/reschedule",
        "json_arguments": {
            "first name": None,
            "last name": None,
            "reason_for_reschedule": None,
            "new_date": None,
            "new_date_end": None,
        },
        "json_arguments_schema": {
            "first name": "str",
            "last name": "str",
            "reason_for_visit": "str",
            "date": "datetime.date",
            "time_start": "datetime.time",
            "duration": "datetime.time",
        },
        "json_arguments_options": {
            "first name": None,
            "last name": None,
            "reason_for_reschedule": None,
            "new_date": None,
            "new_date_end": None,
        },
        "json_arguments_expected_format": {
            "first name": "John",
            "last name": "Doe",
            "reason_for_visit": "checkup",
            "date": "2024-07-23",
            "time_start": "09:00:00",
            "duration": "00:30:00"
        }
    },
    "provider_availability_api": {
        "api_name": "check provider availability",
        "api_path": "https://hetzner-cpu-1.healiom-service.com/backend/dev/api/v1/provider_availability",
        "json_arguments": {
            "start_date": None,
            "number_of_days": None,
            "start_time": None,
            "end_time": None,
        },
        "json_arguments_schema": {
            "start_date": "string",
            "number_of_days": "int",
            "start_time": "string",
            "end_time": "string",
        },
        "json_arguments_options": {
            "start_date": None,
            "number_of_days": None,
            "start_time": None,
            "end_time": None,
        },
        "json_arguments_example": {
            "start_date": "2024-08-02",
            "number_of_days": 7,
            "start_time": "02:30",
            "end_time": "23:00"
        }
    }
}

LOGIC_GRAPH = {
    "start": "PROVIDER_AVAILABILITY_NODE",
    "nodes": {
        "PROVIDER_AVAILABILITY_NODE": {
            "type": "api",
            "api_to_call": AVAILABLE_APIS["provider_availability_api"],
            "next_node": "PROVIDER_AVAILABILITY_NODE_B"
        },
        "PROVIDER_AVAILABILITY_NODE_B": {
            "type": "logic",
            "logic_block": """
if memory['intent'] == "schedule appointment":
    res["next_node"] = "SCHEDULE_APPOINTMENT_NODE"
else:
    res["next_node"] = "RESCHEDULE_APPOINTMENT_NODE"
                """
        },
        "SCHEDULE_APPOINTMENT_NODE" : {
            "type": "api",
            "api_to_call": AVAILABLE_APIS["schedule_api"],
            "next_node": None
        },
        "RESCHEDULE_APPOINTMENT_NODE" : {
            "type": "api",
            "api_to_call": AVAILABLE_APIS["reschedule_api"],
            "next_node": None
        }
    }
}

INTENTS = [
    {
        "name": "schedule appointment",
        "alt_names": ["make a visit"],
        "logic_graph": {
            "start": "PROVIDER_AVAILABILITY_NODE",
            "nodes": {
                "PROVIDER_AVAILABILITY_NODE": {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["provider_availability_api"],
                    "next_node": "PROVIDER_AVAILABILITY_NODE_B"
                },
                "PROVIDER_AVAILABILITY_NODE_B": {
                    "type": "logic",
                    "logic_block": """
if memory['intent'] == "schedule appointment":
    res["next_node"] = "SCHEDULE_APPOINTMENT_NODE"
else:
    res["next_node"] = "RESCHEDULE_APPOINTMENT_NODE"
                    """
                },
                "SCHEDULE_APPOINTMENT_NODE" : {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["schedule_api"],
                    "next_node": None
                },
                "RESCHEDULE_APPOINTMENT_NODE" : {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["reschedule_api"],
                    "next_node": None
                }
            }
        }
    },
    {
        "name": "reschedule appointment",
        "alt_names": ["reschedule visit"],
        "logic graph": {
            "start": "PROVIDER_AVAILABILITY_NODE",
            "nodes": {
                "PROVIDER_AVAILABILITY_NODE": {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["provider_availability_api"],
                    "next_node": "PROVIDER_AVAILABILITY_NODE_B"
                },
                "PROVIDER_AVAILABILITY_NODE_B": {
                    "type": "logic",
                    "logic_block": """
if memory['intent'] == "schedule appointment":
    res["next_node"] = "SCHEDULE_APPOINTMENT_NODE"
else:
    res["next_node"] = "RESCHEDULE_APPOINTMENT_NODE"
                    """
                },
                "SCHEDULE_APPOINTMENT_NODE" : {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["schedule_api"],
                    "next_node": None
                },
                "RESCHEDULE_APPOINTMENT_NODE" : {
                    "type": "api",
                    "api_to_call": AVAILABLE_APIS["reschedule_api"],
                    "next_node": None
                }
            }
        }
    },
]

In [64]:
def which_next(logic_block, memory):
    res ={}
    
    exec(logic_block)
    
    next_api = res["next_node"]

    if next_api:
        return next_api
    else:
        return None  


next_node = which_next(LOGIC_GRAPH["nodes"]["PROVIDER_AVAILABILITY_NODE_B"]["logic_block"], CONTEXT)

print(next_node)

SCHEDULE_APPOINTMENT_NODE


In [66]:
def execution_loop(logic_graph, context):
    next_node = logic_graph["start"]
    while next_node:
        node = logic_graph["nodes"][next_node]
        if node["type"] == "api":
            api_to_call = node["api_to_call"]
            print(f"calling API: {api_to_call['api_name']}")
            print(f"API path: {api_to_call['json_arguments']}")
            next_node = node["next_node"]
        elif node["type"] == "logic":
            next_node = which_next(node["logic_block"], context)
        else:
            print("Invalid node type")
            break

execution_loop(LOGIC_GRAPH, CONTEXT)

calling API: check provider availability
API path: {'start_date': None, 'number_of_days': None, 'start_time': None, 'end_time': None}
calling API: schedule visit
API path: {'first name': None, 'last name': None, 'reason_for_visit': None, 'date': None, 'time_start': None, 'duration': None, 'provider': None}


In [62]:
# execution loop using intents instead of logic graph
def execution_loop_using_intents(intents, context):
    for intent in intents:
        if context["intent"] in intent["alt_names"]:
            logic_graph = intent["logic_graph"]
            next_node = logic_graph["start"]
            while next_node:
                node = logic_graph["nodes"][next_node]
                if node["type"] == "api":
                    api_to_call = node["api_to_call"]
                    print(f"calling API: {api_to_call['api_name']}")
                    next_node = node["next_node"]
                elif node["type"] == "logic":
                    next_node = which_next(node["logic_block"], context)
                else:
                    print("Invalid node type")
                    break
            break

execution_loop_using_intents(INTENTS, CONTEXT)

# filldata

In [70]:
def fill_data_from_context(context, data, debug=False):
    if debug:
        print(f"fill_data_from_context called with context: {context}, data: {data}")
    
    data_json_string = json.dumps(data, indent=2)

    llm_prompt = f"""
    Given the following data payload, see if you can extract any fields from the context that are missing in the data. If you find any, return them in dictionary form.
    For example, if the context contains "My name is John and I want to schedule an appointment for a checkup" and the data is  
    {{
            "first name": None,
            "last name": None,
            "reason_for_visit": None,
            "date": None,
            "time_start": None,
            "duration": None,
        }}
    then you should return {{"first name": "John", 
                            "reason_for_visit": "checkup"
                            }}
    Here is the data payload:
    {data_json_string}
    """

    llm_output =  llm.invoke(llm_prompt).content.strip()

    llm_output = llm_output.strip()
    
    if not llm_output:
        return data

    try:
        extracted_data = json.loads(llm_output)

        for key in extracted_data:
            data[key] = extracted_data[key]
    except json.JSONDecodeError as e:
        if debug:
            print(f"Error extracting data from context: {e}")
        pass
    except Exception as e:
        if debug:
            print(f"Unexpected error: {e}")

    if debug:
        print(f"Data after filling from context: {data}")

    return data


# construct question

In [22]:
def construct_question(json_context: dict, target_variable: str, hint: str = None, output_type: str = None, output_options: list = None, debug=False):
    if debug:
        print(f"construct_question called with json_context, (conversation) target_variable: '{target_variable}', output_type: '{output_type}', output_options: {output_options}")
    context_str = json.dumps(json_context, indent=2)

    prompt = f"Given the following context:\n{context_str}\n\n"
    prompt += f"Please construct a question to ask the user to obtain the value for '{target_variable}'"

    if hint:
        prompt += f"Here is additional context on how you should handle asking this question: '{hint}'"
    
    if output_type:
        prompt += f" that is of type '{output_type}'"
    
    if output_options:
        options_str = ', '.join(output_options)
        prompt += f" and should be one of the following options: {options_str}"
        prompt += "Please list the options for the user in the question. One on each line and number them."
    
    prompt += "."

    response = llm.invoke(prompt).content

    if debug:
        print(f"Constructed question: {response}")

    return response.strip()

In [29]:
# make a list of 20 random names
names = ["John", "Jane", "Michael", "Sarah", "David", "Emily", "Robert", "Emma", "William", "Olivia", "James", "Ava", "Daniel", "Sophia", "Benjamin", "Isabella", "Mason", "Mia", "Ethan", "Charlotte"]

res = construct_question(json_context=CONTEXT, target_variable="provider", hint=None, output_type="str", output_options=names, debug=True)


construct_question called with json_context, (conversation) target_variable: 'provider', output_type: 'str', output_options: ['John', 'Jane', 'Michael', 'Sarah', 'David', 'Emily', 'Robert', 'Emma', 'William', 'Olivia', 'James', 'Ava', 'Daniel', 'Sophia', 'Benjamin', 'Isabella', 'Mason', 'Mia', 'Ethan', 'Charlotte']
Constructed question: Which provider would you like to schedule an appointment with?
1. John
2. Jane
3. Michael
4. Sarah
5. David
6. Emily
7. Robert
8. Emma
9. William
10. Olivia
11. James
12. Ava
13. Daniel
14. Sophia
15. Benjamin
16. Isabella
17. Mason
18. Mia
19. Ethan
20. Charlotte


In [79]:
def interpret_response(original_question: str, output_data_type: str, output_options, user_response: str, debug=False):
    if debug:
        print(f"interpret_response called with original_question: '{original_question}', output_data_type: '{output_data_type}', output_options: {output_options}, user_response: '{user_response}'")
    if output_options is None:
        output_options = "Can be anything"
    else:
        output_options = ', '.join(map(str, output_options))

    if output_data_type == "datetime.time" or  output_data_type == "datetime.date":
        data_validation_response = user_response
    
    else:
        prompt = (
            f"Given the following question and user response, please extract the relevant value.\n\n"
            f"Question: {original_question}\n"
            f"Expected data type: {output_data_type}\n"
            f"Possible options: {output_options}\n"
            f"User response: {user_response}\n\n"
            f"Please provide the extracted value (if possible options are given, make sure that the value exists in the possible options or return 'Not applicable'):"
        )
        
        response = llm.invoke(prompt).content.strip()

        if debug:
            print(f"LLM response for interpret_response: {response}")

        data_validation_prompt = f""" An LLM has been given a task of data extraction from a string. Your job is to verify the data was extracted correctly.
        You will also be given a data type and a list of possible options for the data (can be None). Make sure that your response fits the data type and is in one of the options.
        Here is the prompt to the LLM: {prompt}
        Here is the response from the LLM: {response}
        Here is the data type: {output_data_type}
        Here are the possible options: {output_options}
        Please make sure that the response is correct and fits the data type and is in the possible options. You MUST provide a output that matches the data type exactly.
        """
        
        data_validation_response = llm.invoke(data_validation_prompt).content.strip()

    if debug:
        print(f"Data validation response: {data_validation_response}")

    try:
        value = cast_value(output_data_type, data_validation_response, debug=debug)
        if debug:
            print(f"Interpreted value: {value}")
        return value
    except Exception as e:
        if debug:
            print(f"Error in interpret_response: {e}")
        return False

In [84]:
s = """Which provider would you like to schedule an appointment with?
1. John
2. Jane
3. Michael
4. Sarah
5. David
6. Emily
7. Robert
8. Emma
9. William
10. Olivia
11. James
12. Ava
13. Daniel
14. Sophia
15. Benjamin
16. Isabella
17. Mason
18. Mia
19. Ethan
20. Charlotte"""

res = interpret_response(original_question=s, output_data_type="str", output_options=names, user_response="12", debug=False)

print(res)

Ava


# provider API

In [3]:
import requests
import os

# Define the endpoint URL
url = "https://hetzner-cpu-1.healiom-service.com/backend/dev/api/v1/provider_availability"

# Define the headers including the bearer token
headers = {
    "Authorization": f"Bearer {os.getenv('JWT_TOKEN')}",
    "he_type": "Patient",
    "organization_id": "776e17a2-f7cb-45b0-a52a-630e57c7227e"
}

# Define the data payload
data = {
    "start_date": "2024-08-02",
    "number_of_days": 7,
    "start_time": "02:30",
    "end_time": "23:00"
}

# Make the POST request to the API
response = requests.post(url, headers=headers, json=data)

print(os.getenv('JWT_TOKEN'))

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    print(response.json())
    # names =[]
    # for item in response.json()["items"]:
    #     if 'display_name' in item:
    #         names.append(item['display_name'])
    # print(names)
    resp = response.json()

else:
    print(f"Request failed with status code {response.status_code}")
    print("Response:", response.text)


eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJwYXlsb2FkIjoiTmMyL1dlYmxMNERMME83ZWxDUHRteEVXSDZsK2QvbUxHWUFiUTB6RC9RTzZrUWx5bklkREdJclo2MWRBQ3pXK1dORHFTSmxsdUt0TTgvUGx6Sk5uU0c3Nk5oMDd3VTBZMFhtRmFjR0NzTnZpZUpTOExuOC9BTkhCZnZuTG5rQ3p4MjJUdkxBeE4wODNYb3o0ZUNBUzJSendyNThVMTNmbUVSNlhyVVVMNDNnSkJPSTVpWndQWHFkdFFlSU1rRkd1UVd5eUM5eFdzVEkycXBSZ2cvUXpsNzJDTGdjbFNiZ0ZYYVRLSy96SU0xNUFjRGt3YTNlQ0crVjkvOVNyOUUxL3g4OGtQOEVSa2dWbXo4Qy9ZbzczUzhOektQNUVGcXRNRDczK1AxVWFHanp3bms1SGsvcnA0YWdtcUhLSXVjY1JjM2RnUFYxK3JJQVRkYSsxWVdlYUxDN1lGT0ZlUHlWTEhlSjBvVGV0Y3hNVnovR0hWSXpCdXdRVUREQ3hSY3R3NmRxdjFuQzZXZmI5YmQza0RPa3Z3Tmh5cWI4MlFPMXBHVEgwb1hUUkk1dVFXd2NvUUtybk5lMW82RVV3K0VBN2hwckVrM0NiYWUyMERvSmQ0Y3U5RktnbEx1MzRzeHJONnJnRnBNaFlhaXYzeis3NDB2WjJqc0FtcUdIZkNLNTUyWGxSKzVuaFVDMVNJV2Z3TUh3cmNpT1U1aERWNzBiQ2hCV3MvaC9lY2ZlZ2ZqM3ZWSlF0RDJJWXNuNTdOSCs0ekFIK3lYUXpoWmttMmRBamJvS3o2cTRhZW80SHFSS3VIZzlVR3ZzYnNIZzBkVFkxaUovK09DUUZ5cEhmbVB1Si94VEhjanBBb3pMcUpoL3lTeU5STGV2WXJMMjIxSVJWYXBhWVFYajV0OVZVd1liZko3K1J0alhHOTBMMUNwbE00UHBCOTR

In [8]:
d = {"available_date_time": {
                "2024-08-02": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-03": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-04": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-05": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-06": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-07": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ],
                "2024-08-08": [
                    {
                        "start_time": "02:30",
                        "end_time": "23:00"
                    }
                ]
            }
}

def extract_available_dates(resp):
    available_dates = []
    for date in resp["available_date_time"]:
        available_dates.append(date)
    return available_dates

dates = extract_available_dates(d)

print(dates)

['2024-08-02', '2024-08-03', '2024-08-04', '2024-08-05', '2024-08-06', '2024-08-07', '2024-08-08']


In [13]:
availability_by_date = {}

for provider in resp["items"]:
    avialble_date_time = provider["available_date_time"]
    for date in avialble_date_time:
        if date not in availability_by_date:
            availability_by_date[date] = 1
        else:
            availability_by_date[date] += 1

print(availability_by_date)

{'2024-08-02': 26, '2024-08-03': 26, '2024-08-04': 26, '2024-08-05': 26, '2024-08-06': 26, '2024-08-07': 26, '2024-08-08': 26}


# Extras

In [32]:
s = """
To obtain the value for 'provider', please choose one of the following options by typing the corresponding number:
1. Rishabh Shah
2. Yii Zhang
3. Brad Pitt
4. Michael L Gales
5. Shubham Patidar
6. Karunyan Arulanantham
7. Florian Brody
8. Shawn Cole
9. Gayathri Krishna Kumar
10. Arnold Bays
11. Christiano Ronaldo
12. Laura Koue FNP-C
13. Sean Howse
14. Carla Gallegos
15. Veena Damle
16. Manish Asodekar
17. Test34 Iijkjlk
18. Test Dfasdf
19. Test Test
20. Chaitrali Kharose
21. Chetana Patil
22. Thomas Yarema
23. Rakesh Nemade
24. Balu B
25. Pradeep Damle

"""

print(s.strip())

To obtain the value for 'provider', please choose one of the following options by typing the corresponding number:
1. Rishabh Shah
2. Yii Zhang
3. Brad Pitt
4. Michael L Gales
5. Shubham Patidar
6. Karunyan Arulanantham
7. Florian Brody
8. Shawn Cole
9. Gayathri Krishna Kumar
10. Arnold Bays
11. Christiano Ronaldo
12. Laura Koue FNP-C
13. Sean Howse
14. Carla Gallegos
15. Veena Damle
16. Manish Asodekar
17. Test34 Iijkjlk
18. Test Dfasdf
19. Test Test
20. Chaitrali Kharose
21. Chetana Patil
22. Thomas Yarema
23. Rakesh Nemade
24. Balu B
25. Pradeep Damle


In [35]:
s = "09:00:00"

# convert s to datetime.time
import datetime

v = datetime.datetime.strptime(s, "%H:%M:%S").time()

s = "2024-07-23"

# convert s to datetime.date
v = datetime.datetime.strptime(s, "%Y-%m-%d").date()

v

datetime.time(9, 0)